In [2]:
import sys,os,signal

import numpy as np
import musicnet

import torch
from torch.autograd import Variable
from torch.nn.functional import conv1d, mse_loss

from time import time

import matplotlib.pyplot as plt
from IPython.display import Audio

from sklearn.metrics import average_precision_score

root = './'

%matplotlib inline

In [3]:
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'   # see issue #152
os.environ['CUDA_VISIBLE_DEVICES']='2'

def worker_init(args):
    signal.signal(signal.SIGINT, signal.SIG_IGN) # ignore signals so parent can handle them
    np.random.seed(os.getpid() ^ int(time())) # approximately random seed for workers

batch_size = 100
kwargs = {'num_workers': 4, 'pin_memory': True, 'worker_init_fn': worker_init}

m = 128
k = 500 # number of frequency bins?
d = 4096 # size of windows?
window = 16384 # total number of audio samples?
stride = 512
regions = 1 + (window - d)//stride # number of output after sliding the window through x

In [4]:
train_set = musicnet.MusicNet(root=root, train=True, download=True, window=window)#, pitch_shift=5, jitter=.1)
test_set = musicnet.MusicNet(root=root, train=False, window=window, epoch_size=50000)

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_size,**kwargs)
test_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=batch_size,**kwargs)

In [15]:
a = train_set.rec_ids
b = test_set.rec_ids

In [14]:
len(a)

320

In [16]:
len(b)

10

In [15]:
def create_filters(d,k,low=50,high=6000):
    x = np.linspace(0, 2*np.pi, d, endpoint=False)
    wsin = np.empty((k,1,d), dtype=np.float32)
    wcos = np.empty((k,1,d), dtype=np.float32)
    start_freq = low
    end_freq = high
    num_cycles = start_freq*d/44000.
    scaling_ind = np.log(end_freq/start_freq)/k
    window_mask = 1.0-1.0*np.cos(x)
    for ind in range(k):
        wsin[ind,0,:] = window_mask*np.sin(np.exp(ind*scaling_ind)*num_cycles*x)
        wcos[ind,0,:] = window_mask*np.cos(np.exp(ind*scaling_ind)*num_cycles*x)
    
    return wsin,wcos

In [16]:
wsin,wcos = create_filters(d,k)

with torch.cuda.device(0):
    wsin_var = Variable(torch.from_numpy(wsin).cuda(), requires_grad=False)
    wcos_var = Variable(torch.from_numpy(wcos).cuda(), requires_grad=False)
    
def init_weights(scale=0.):
    wscale = 0.
    with torch.cuda.device(0):
        beta = Variable(wscale*torch.rand([regions*k,m]).cuda(), requires_grad=True)
        betaavg = Variable(beta.data.clone(), requires_grad=False)
    return (beta,), (betaavg,) # pytorch optimizer requires iterable 

def forward(x, beta):
    zx = conv1d(x[:,None,:], wsin_var, stride=stride).pow(2) \
       + conv1d(x[:,None,:], wcos_var, stride=stride).pow(2)
    return torch.mm(torch.log(zx + musicnet.epsilon).view(x.data.size()[0],regions*k), beta)

def L(y_hat, y, beta):
    # adjust for per-frame loss
    return mse_loss(y_hat, y)*128/2.# + .01*torch.mean(beta.pow(2.))

# Training the model

In [17]:
weights, averages = init_weights()
loss_history = []
avgp_history = []

In [18]:
avg = .9998
optimizer = torch.optim.SGD(weights, lr=0.000001, momentum=.95)

In [19]:
try:
    with train_set, test_set:
        print("squre loss\tabg prec\ttime\t\tutime")
        for epoch in range(50):
            t = time()
            for i, (x,y) in enumerate(train_loader):
                optimizer.zero_grad()
                
                # making x and y into pytorch dealable format
                x = Variable(x.cuda(), requires_grad=False) 
                y = Variable(y.cuda(), requires_grad=False) 
                
                loss = L(forward(x, *weights),y , *weights)
                loss.backward()
                optimizer.step()
                
                for w, wavg in zip(weights, averages):
                    wavg.data.copy_(avg*wavg.data + (1.-avg)*w.data) # what happens if I change to equal
                    
            t1 = time()
            avgp, loss = 0.,0.
            
            # For testing
            yground = torch.FloatTensor(batch_size*len(test_loader), m) # what not do this together with loss
            yhat = torch.FloatTensor(batch_size*len(test_loader), m)
            for i, (x_test,y_test) in enumerate(test_loader):
                x_test = Variable(x_test.cuda(), requires_grad=False)
                y_test = Variable(y_test.cuda(), requires_grad=False)
                yhatvar = forward(x_test, *averages)
                loss += L(yhatvar, y_test, *averages).item()
                yground[i*batch_size:(i+1)*batch_size,:] = y_test.data
                yhat[i*batch_size:(i+1)*batch_size,:] = yhatvar.data
            avgp = average_precision_score(yground.numpy().flatten(),yhat.numpy().flatten())
#             avgp = average_precision_score(yground.numpy().flatten(),yhat.numpy().flatten())
            loss_history.append(loss/len(test_loader))
            avgp_history.append(avgp)
            print('{:2f}\t{:2f}\t{:2f}\t{:2f}'.format(loss_history[-1],avgp_history[-1],time()-t, time()-t1))
            
            
except KeyboardInterrupt:
    print('Graceful Exit')
else:
    print("Finsihed")

squre loss	abg prec	time		utime
0.984708	0.621846	21.497175	8.801307
0.871883	0.634628	21.538096	8.766026
0.804300	0.643566	20.522870	7.900976
0.753901	0.648922	19.113822	7.779321
0.730231	0.652340	19.604584	7.766824
0.708090	0.653427	19.539460	7.722400
0.694748	0.655970	19.503912	8.096855
0.689197	0.656575	19.453594	7.751936
0.684582	0.661932	20.043772	8.111955
0.684816	0.657823	19.406796	8.070664
0.683510	0.660495	19.623866	7.859873
0.683197	0.659542	19.459719	7.620090
0.681832	0.661562	19.428283	7.920199
0.686717	0.661191	19.590956	7.887896
0.685196	0.661920	19.752652	8.021381
0.684777	0.661020	19.524984	7.983444
0.686127	0.662452	20.298046	8.501631
0.686875	0.665172	19.712829	8.238118
0.683044	0.663084	19.557666	7.909143
0.689059	0.661943	19.628227	7.946543
0.686240	0.662614	19.688740	8.060700
0.682640	0.666309	19.582536	7.914261
0.691255	0.663827	20.199381	8.413030
0.688870	0.667830	19.721585	8.141144
0.688653	0.664553	19.592653	7.972461
0.684856	0.666406	19.634136	7.807541
0.6853